# HiPlot in Notebooks

A simple guide to get HiPlot working for hyperparameter tuning

https://github.com/facebookresearch/hiplot


## HiPlot + Optuna + PyTorch (Hyperparameter Optimisation Techniques and displays)

Using Optuna to perform hyperparameter optimisation on a MLP Pytorch Model, then visualising the results using HiPlot

In [1]:
# install hiplot library for visualisation
!pip install -U hiplot
# install optuna library for hyperparameter optimisation
!pip install optuna

     |████████████████████████████████| 757kB 8.5MB/s 
     |████████████████████████████████| 788kB 10.2MB/s 
     |████████████████████████████████| 358kB 24.1MB/s 
     |████████████████████████████████| 358kB 17.4MB/s 
ERROR: jupyter-console 5.2.0 has requirement prompt-toolkit<2.0.0,>=1.0.0, but you'll have prompt-toolkit 3.0.8 which is incompatible.
ERROR: google-colab 1.0.0 has requirement ipython~=5.5.0, but you'll have ipython 7.16.1 which is incompatible.
  Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      Successfully uninstalled prompt-toolkit-1.0.18
  Found existing installation: ipython 5.5.0
    Uninstalling ipython-5.5.0:
      Successfully uninstalled ipython-5.5.0


     |████████████████████████████████| 266kB 9.7MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 163kB 13.8MB/s 
     |████████████████████████████████| 81kB 6.9MB/s 
     |████████████████████████████████| 81kB 8.1MB/s 
     |████████████████████████████████| 133kB 14.0MB/s 
     |████████████████████████████████| 51kB 7.0MB/s 
     |████████████████████████████████| 112kB 16.1MB/s 
  Created wheel for optuna: filename=optuna-2.3.0-cp36-none-any.whl size=359761 sha256=015d9710bfcd5ddd6194c95768200be3f37767e4c06606d40e288b78410129bb
  Stored in directory: /root/.cache/pip/wheels/fa/91/19/64b0ec6b964f89c0695a9dc6db6f851d0b54c5381a5c9cadfb
Successfully built optuna
  Created wheel for PrettyTable: filename=prettytable-0.7.2-cp36-none-any.whl size=13702 sha256=13b411763772126ba92c6b7f70b568b6e5d30e7a0ed1ff1cf6b06155aed9fb69
  Stored in directory: /root/.cache/pip/whe

In [2]:
import hiplot as hip

In [3]:
#@title Hyperparameter tuning using PyTorch, Optuna {display-mode: "form"}
#@markdown Using MNIST dataset, optimise a PyTorch Model (MLP) to improve accuracy of hand-written digit recognition, taken from https://github.com/optuna/optuna/blob/master/examples/pytorch_simple.py


# This code will be hidden when the notebook is loaded.

"""
Optuna example that optimizes multi-layer perceptrons using PyTorch.
In this example, we optimize the validation accuracy of hand-written digit recognition using
PyTorch and MNIST. We optimize the neural network architecture as well as the optimizer
configuration. As it is too time consuming to use the whole MNIST dataset, we here use a small
subset of it.
"""

import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from torchvision import datasets
from torchvision import transforms

import optuna


DEVICE = torch.device("cpu")
BATCHSIZE = 128
CLASSES = 10
DIR = os.getcwd()
EPOCHS = 10
LOG_INTERVAL = 10
N_TRAIN_EXAMPLES = BATCHSIZE * 30
N_VALID_EXAMPLES = BATCHSIZE * 10


def define_model(trial):
    # HiPlot - No optimisation of number layers,
    # n_layers = trial.suggest_int("n_layers", 1, 3)
    n_layers = 3
    layers = []

    in_features = 28 * 28
    for i in range(n_layers):
        out_features = trial.suggest_int("n_units_l{}".format(i), 4, 128)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        p = trial.suggest_float("dropout_l{}".format(i), 0.2, 0.5)
        layers.append(nn.Dropout(p))

        in_features = out_features
    layers.append(nn.Linear(in_features, CLASSES))
    layers.append(nn.LogSoftmax(dim=1))

    return nn.Sequential(*layers)


def get_mnist():
    # Load MNIST dataset.
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST(DIR, train=True, download=True, transform=transforms.ToTensor()),
        batch_size=BATCHSIZE,
        shuffle=True,
    )
    valid_loader = torch.utils.data.DataLoader(
        datasets.MNIST(DIR, train=False, transform=transforms.ToTensor()),
        batch_size=BATCHSIZE,
        shuffle=True,
    )

    return train_loader, valid_loader


def objective(trial):

    # Generate the model.
    model = define_model(trial).to(DEVICE)

    # Generate the optimizers.
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

    # Get the MNIST dataset.
    train_loader, valid_loader = get_mnist()

    # Training of the model.
    for epoch in range(EPOCHS):
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            # Limiting training data for faster epochs.
            if batch_idx * BATCHSIZE >= N_TRAIN_EXAMPLES:
                break

            data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)

            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()

        # Validation of the model.
        model.eval()
        correct = 0
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(valid_loader):
                # Limiting validation data.
                if batch_idx * BATCHSIZE >= N_VALID_EXAMPLES:
                    break
                data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)
                output = model(data)
                # Get the index of the max log-probability.
                pred = output.argmax(dim=1, keepdim=True)
                correct += pred.eq(target.view_as(pred)).sum().item()

        accuracy = correct / min(len(valid_loader.dataset), N_VALID_EXAMPLES)

        trial.report(accuracy, epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return accuracy

In [4]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100, timeout=600)

pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))


[I 2020-12-29 12:58:09,341] A new study created in memory with name: no-name-febbc74c-4be2-479d-a052-63ab4e285ccc


Extracting /content/MNIST/raw/train-images-idx3-ubyte.gz to /content/MNIST/raw


Extracting /content/MNIST/raw/train-labels-idx1-ubyte.gz to /content/MNIST/raw



Extracting /content/MNIST/raw/t10k-images-idx3-ubyte.gz to /content/MNIST/raw


Extracting /content/MNIST/raw/t10k-labels-idx1-ubyte.gz to /content/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning:

The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)



[I 2020-12-29 12:58:18,934] Trial 0 finished with value: 0.90390625 and parameters: {'n_units_l0': 110, 'dropout_l0': 0.4836728228193914, 'n_units_l1': 20, 'dropout_l1': 0.41140888100563416, 'n_units_l2': 54, 'dropout_l2': 0.3223606449227492, 'optimizer': 'Adam', 'lr': 0.0015796953079710715}. Best is trial 0 with value: 0.90390625.
[I 2020-12-29 12:58:23,406] Trial 1 finished with value: 0.42109375 and parameters: {'n_units_l0': 18, 'dropout_l0': 0.27833853286383514, 'n_units_l1': 14, 'dropout_l1': 0.40999449958859474, 'n_units_l2': 5, 'dropout_l2': 0.2646475646866994, 'optimizer': 'RMSprop', 'lr': 0.00024158227667283385}. Best is trial 0 with value: 0.90390625.
[I 2020-12-29 12:58:28,303] Trial 2 finished with value: 0.30234375 and parameters: {'n_units_l0': 19, 'dropout_l0': 0.4959594057885399, 'n_units_l1': 59, 'dropout_l1': 0.2369056300546472, 'n_units_l2': 4, 'dropout_l2': 0.27327414114341636, 'optimizer': 'Adam', 'lr': 0.04808443449088103}. Best is trial 0 with value: 0.90390625.

Study statistics: 
  Number of finished trials:  100
  Number of pruned trials:  63
  Number of complete trials:  37
Best trial:
  Value:  0.94921875
  Params: 
    n_units_l0: 114
    dropout_l0: 0.3281262795464433
    n_units_l1: 67
    dropout_l1: 0.31844151838247703
    n_units_l2: 101
    dropout_l2: 0.40832913858419356
    optimizer: Adam
    lr: 0.004756481949923075


In [5]:
hyper_opt_data = []
for each_trial in study.trials:
    trial_params = each_trial.params.copy()
    trial_params["uid"] = each_trial.number
    trial_params["state"] = str(each_trial.state)
    trial_params["accuracy"] = each_trial.value
    hyper_opt_data.append(trial_params)
hip.Experiment.from_iterable(hyper_opt_data).display()

<IPython.core.display.Javascript object>